# Evaluación de Modelos LLM en Modismos Colombianos

In [1]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Estilos para gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Modelos
MODELS_FILE = 'LLMs_Results/models.txt'

def load_models_from_file(filepath):
    """Carga los nombres de modelos desde un archivo de texto."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {filepath}")
        return []

MODEL_NAMES = load_models_from_file(MODELS_FILE)

# Directorios
DATA_DIR = 'LLMs_Results'
OUTPUT_DIR = 'Metrics_Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Datos en: {DATA_DIR}")
print(f"Resultados en: {OUTPUT_DIR}")
print(f"Modelos: {MODEL_NAMES}")

Datos en: LLMs_Results
Resultados en: Metrics_Results
Modelos: ['amazon/nova-micro-v1', 'microsoft/phi-4', 'amazon/nova-lite-v1', 'cohere/command-r-08-2024', 'qwen/qwen-2.5-72b-instruct', 'google/gemma-2-27b-it', 'meta-llama/llama-3.3-70b-instruct', 'microsoft/wizardlm-2-8x22b', 'meta-llama/llama-4-maverick', 'qwen/qwen2.5-vl-32b-instruct:free', 'x-ai/grok-3-mini-beta', 'perplexity/sonar', 'mistralai/mistral-medium-3', 'mistralai/mixtral-8x7b-instruct', 'google/gemini-2.5-flash', 'meta-llama/llama-3.1-405b-instruct', 'deepseek/deepseek-chat-v3.1', 'moonshotai/kimi-k2-0905', 'openai/o4-mini-high', 'openai/gpt-4.1', 'openai/o1-mini', 'anthropic/claude-sonnet-4', 'gpt-5.1']


In [8]:
# Importar funciones de métricas
import sys
sys.path.append('CodeMetrics')

from BertScore import compute_bertscore_beto, compute_bertscore_sci_beto
from SentenceBert import compute_sbert_similarity, compute_xlm_similarity, compute_scibeto_similarity
from chrF import compute_chrf_batch

print("✓ Métricas cargadas correctamente:")
print("  - BERTScore (BETO y SciBETO)")
print("  - Sentence-BERT (paraphrase-multilingual-mpnet-base-v2, stsb-xlm-r-multilingual, scibert-mean)")
print("  - chrF (Character n-gram F-score)")

✓ Métricas cargadas correctamente:
  - BERTScore (BETO y SciBETO)
  - Sentence-BERT (paraphrase-multilingual-mpnet-base-v2, stsb-xlm-r-multilingual, scibert-mean)
  - chrF (Character n-gram F-score)


## 1. PROMPT 1: Modismo → Es Modismo (Sí/No)

In [ ]:
GROUNTH_TRUTH = 6_533

In [ ]:
print("="*80)
print("PROMPT 1: Modismo → ¿Es Modismo? (Sí/No)")
print("="*80)

print("Objetivo: Evaluar si el modelo identifica correctamente que una expresión es un modismo")
print("Métrica: Accuracy (Exactitud)")
print("Justificación: Problema de clasificación binaria (Sí/No).")
print("   Mide el porcentaje de respuestas correctas.")
print("   Los errores/omisiones se cuentan como falsos negativos.")
print("-"*80)

# Cargar datos desde JSON
with open(os.path.join(DATA_DIR, 'prompt_1_metrics_data.json'), 'r', encoding='utf-8') as f:
    data_p1 = json.load(f)

# Filtrar datos vacíos
data_p1_valid = [d for d in data_p1 if d.get('es_modismo_real') and d.get('es_modismo_generado')]
print(f"Datos cargados: {len(data_p1_valid)} registros válidos de {len(data_p1)} totales")
print()

def normalizar_respuesta(respuesta):
    """Normaliza respuestas a 'Sí' o 'No'"""
    if not respuesta:
        return None
    respuesta = str(respuesta).strip().lower()
    if respuesta in ['sí', 'si', 'yes', 's', 'true', '1']:
        return 'Sí'
    elif respuesta in ['no', 'n', 'false', '0']:
        return 'No'
    return respuesta

resultados_p1 = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")

    # Filtrar datos de este modelo
    model_data = [d for d in data_p1_valid if d.get('modelo') == model]

    if not model_data:
        print(f"  ⚠ No hay datos para {model} - todos los registros cuentan como errores")
        # Agregar todos los registros como falsos negativos
        for _ in range(GROUNTH_TRUTH):
            resultados_p1.append({
                'modismo': 'N/A',
                'modelo': model,
                'respuesta_real': 'Sí',
                'respuesta_generada': 'No',
                'correcto': False
            })
        print(f"  • Accuracy: 0.0000 (0/{GROUNTH_TRUTH} correctos)")
        continue

    # Calcular accuracy sobre los datos procesados
    correctos = 0
    for d in model_data:
        real = normalizar_respuesta(d['es_modismo_real'])
        generado = normalizar_respuesta(d['es_modismo_generado'])
        correcto = (real == generado)

        if correcto:
            correctos += 1

        resultados_p1.append({
            'modismo': d['modismo'],
            'modelo': model,
            'respuesta_real': real,
            'respuesta_generada': generado,
            'correcto': correcto
        })

    # Calcular registros faltantes (omitidos por errores)
    registros_procesados = len(model_data)
    registros_faltantes = GROUNTH_TRUTH - registros_procesados

    # Agregar registros faltantes como falsos negativos
    if registros_faltantes > 0:
        print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan como incorrectos")
        for _ in range(registros_faltantes):
            resultados_p1.append({
                'modismo': 'ERROR/OMITIDO',
                'modelo': model,
                'respuesta_real': 'Sí',
                'respuesta_generada': 'No',
                'correcto': False
            })

    # Calcular accuracy sobre el total (GROUNTH_TRUTH)
    accuracy = correctos / GROUNTH_TRUTH
    print(f"  • Accuracy: {accuracy:.4f} ({correctos}/{GROUNTH_TRUTH} correctos)")
    print(f"    - Procesados: {registros_procesados}")
    print(f"    - Correctos: {correctos}")
    print(f"    - Incorrectos procesados: {registros_procesados - correctos}")
    print(f"    - Errores/omitidos: {registros_faltantes}")

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_1_accuracy_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_p1, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_p1 if r['modelo'] == model]
    if model_results:
        correctos = sum(1 for r in model_results if r['correcto'])
        total = len(model_results)
        accuracy = correctos / GROUNTH_TRUTH  # Siempre sobre el ground truth
        print(f"   {model}:")
        print(f"      • Accuracy:  {accuracy:.4f} ({correctos}/{GROUNTH_TRUTH} correctos)")

print()

print(f"✓ Guardado en: {output_file}")
print("="*80)
print()


PROMPT 1: Modismo → ¿Es Modismo? (Sí/No)
Objetivo: Evaluar si el modelo identifica correctamente que una expresión es un modismo
Métrica: Accuracy (Exactitud)
Justificación: Problema de clasificación binaria (Sí/No).
   Mide el porcentaje de respuestas correctas.
   Los errores/omisiones se cuentan como falsos negativos.
--------------------------------------------------------------------------------
Datos cargados: 140310 registros válidos de 140310 totales

Evaluando modelo: amazon/nova-micro-v1
  ⚠ 1 registros omitidos se cuentan como incorrectos
  • Accuracy: 0.2464 (1610/6533 correctos)
    - Procesados: 6532
    - Correctos: 1610
    - Incorrectos procesados: 4922
    - Errores/omitidos: 1
Evaluando modelo: microsoft/phi-4
  ⚠ 10 registros omitidos se cuentan como incorrectos
  • Accuracy: 0.5659 (3697/6533 correctos)
    - Procesados: 6523
    - Correctos: 3697
    - Incorrectos procesados: 2826
    - Errores/omitidos: 10
Evaluando modelo: amazon/nova-lite-v1
  ⚠ 3 registros omi

## 2. PROMPT 2: Modismo → Definición

In [ ]:
GROUNTH_TRUTH = 6_533

In [ ]:
import sys
sys.path.append('CodeMetrics') # Ensure CodeMetrics is in path for imports
from SentenceBert import compute_scibeto_similarity # Explicitly import here
from BertScore import compute_bertscore_beto, compute_bertscore_sci_beto # Explicitly import BERTScore functions
from SentenceBert import compute_sbert_similarity, compute_xlm_similarity # And these for consistency
from chrF import compute_chrf_batch

print("="*80)
print("PROMPT 2: Modismo → Definición")
print("="*80)

print("Objetivo: Evaluar si el modelo puede generar una definición correcta del modismo")
print("Métricas:")
print("  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT")
print("  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet")
print("  - chrF: Character n-gram F-score")
print("Nota: Los errores/omisiones se cuentan con score 0 en todas las métricas.")
print("-"*80)

# Cargar datos desde JSON
with open(os.path.join(DATA_DIR, 'prompt_2_metrics_data.json'), 'r', encoding='utf-8') as f:
    data_p2 = json.load(f)

# Filtrar datos vacíos
data_p2_valid = [d for d in data_p2 if d.get('definicion_real') and d.get('definicion_generada')]
print(f"Datos cargados: {len(data_p2_valid)} registros válidos de {len(data_p2)} totales")
print()

# ============================================================================
# 1. BERTScore con BETO y SciBETO
# ============================================================================
bert_models = [
    ('BETO', compute_bertscore_beto),
    ('SciBETO', compute_bertscore_sci_beto)
]

for bert_name, bert_func in bert_models:
    print(f"\n{'='*60}")
    print(f"Calculando BERTScore con {bert_name}")
    print('='*60)

    resultados_p2 = []

    for model in MODEL_NAMES:
        print(f"Evaluando modelo: {model}")

        # Filtrar datos de este modelo
        model_data = [d for d in data_p2_valid if d.get('modelo') == model]

        if not model_data:
            print(f"  ⚠ No hay datos para {model} - todos los registros con score 0")
            # Agregar todos los registros con score 0
            for _ in range(GROUNTH_TRUTH):
                resultados_p2.append({
                    'modismo': 'N/A',
                    'modelo': model,
                    'bert_model': bert_name,
                    'precision': 0.0,
                    'recall': 0.0,
                    'f1_score': 0.0,
                    'definicion_real': '',
                    'definicion_generada': ''
                })
            print(f"  • F1 Score: 0.0000 (0/{GROUNTH_TRUTH})")
        else:
            referencias = [d['definicion_real'] for d in model_data]
            candidatos = [d['definicion_generada'] for d in model_data]

            # BERTScore: compara candidatos (generados) vs referencias (reales)
            P, R, F1 = bert_func(candidatos, referencias)

            for idx, (p, r, f1) in enumerate(zip(P.tolist(), R.tolist(), F1.tolist())):
                resultados_p2.append({
                    'modismo': model_data[idx]['modismo'],
                    'modelo': model,
                    'bert_model': bert_name,
                    'precision': p,
                    'recall': r,
                    'f1_score': f1,
                    'definicion_real': referencias[idx],
                    'definicion_generada': candidatos[idx]
                })

        # Calcular registros faltantes (omitidos por errores)
        registros_procesados = len(model_data)
        registros_faltantes = GROUNTH_TRUTH - registros_procesados

        # Agregar registros faltantes con score 0
        if registros_faltantes > 0:
            print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan con score 0")
            for _ in range(registros_faltantes):
                resultados_p2.append({
                    'modismo': 'ERROR/OMITIDO',
                    'modelo': model,
                    'bert_model': bert_name,
                    'precision': 0.0,
                    'recall': 0.0,
                    'f1_score': 0.0,
                    'definicion_real': '',
                    'definicion_generada': ''
                })

        # Calcular promedios sobre el total (GROUNTH_TRUTH) for the *current* model
        all_f1_scores = [r['f1_score'] for r in resultados_p2 if r['modelo'] == model and r['bert_model'] == bert_name]
        all_precisions = [r['precision'] for r in resultados_p2 if r['modelo'] == model and r['bert_model'] == bert_name]
        all_recalls = [r['recall'] for r in resultados_p2 if r['modelo'] == model and r['bert_model'] == bert_name]

        f1_mean = np.mean(all_f1_scores)
        p_mean = np.mean(all_precisions)
        r_mean = np.mean(all_recalls)

        print(f"  • Precision: {p_mean:.4f}")
        print(f"  • Recall:    {r_mean:.4f}")
        print(f"  • F1 Score:  {f1_mean:.4f}")
        print(f"    - Procesados: {registros_procesados}")
        print(f"    - Errores/omitidos: {registros_faltantes}")

    # Guardar resultados
    output_file = os.path.join(OUTPUT_DIR, f'prompt_2_{bert_name.lower()}_bertscore_resultados.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(resultados_p2, f, ensure_ascii=False, indent=2)

    print()
    print(f"RESULTADOS con {bert_name}:")
    for model in MODEL_NAMES:
        model_results = [r for r in resultados_p2 if r['modelo'] == model]
        if model_results:
            f1_scores = [r['f1_score'] for r in model_results]
            precisions = [r['precision'] for r in model_results]
            recalls = [r['recall'] for r in model_results]

            f1_mean = np.mean(f1_scores)
            f1_std = np.std(f1_scores)
            p_mean = np.mean(precisions)
            r_mean = np.mean(recalls)

            print(f"   {model}:")
            print(f"      • Precision: {p_mean:.4f}")
            print(f"      • Recall:    {r_mean:.4f}")
            print(f"      • F1 Score:  {f1_mean:.4f} (±{f1_std:.4f}) [{len(model_results)}/{GROUNTH_TRUTH}]")

    print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 2. Sentence-BERT Similarity
# ============================================================================
sbert_models = [
    ('SciBETO-mean', compute_scibeto_similarity),
    ('paraphrase-mpnet', compute_sbert_similarity),
    ('XLM-RoBERTa', compute_xlm_similarity)
]

for sbert_name, sbert_func in sbert_models:
    print(f"\n{'='*60}")
    print(f"Calculando Sentence-BERT Similarity con {sbert_name}")
    print('='*60)

    resultados_sbert = []

    for model in MODEL_NAMES:
        print(f"Evaluando modelo: {model}")

        model_data = [d for d in data_p2_valid if d.get('modelo') == model]

        if not model_data:
            print(f"  ⚠ No hay datos para {model} - todos los registros con similarity 0")
            # Agregar todos los registros con similarity 0
            for _ in range(GROUNTH_TRUTH):
                resultados_sbert.append({
                    'modismo': 'N/A',
                    'modelo': model,
                    'sbert_model': sbert_name,
                    'similarity': 0.0,
                    'definicion_real': '',
                    'definicion_generada': ''
                })
            print(f"  • Similitud: 0.0000 (0/{GROUNTH_TRUTH})")
        else:
            referencias = [d['definicion_real'] for d in model_data]
            candidatos = [d['definicion_generada'] for d in model_data]

            # Calcular similitud con Sentence-BERT
            similarities = sbert_func(candidatos, referencias)

            for idx, sim in enumerate(similarities):
                resultados_sbert.append({
                    'modismo': model_data[idx]['modismo'],
                    'modelo': model,
                    'sbert_model': sbert_name,
                    'similarity': float(sim),
                    'definicion_real': referencias[idx],
                    'definicion_generada': candidatos[idx]
                })

        # Calcular registros faltantes (omitidos por errores)
        registros_procesados = len(model_data)
        registros_faltantes = GROUNTH_TRUTH - registros_procesados

        # Agregar registros faltantes con similarity 0
        if registros_faltantes > 0:
            print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan con similarity 0")
            for _ in range(registros_faltantes):
                resultados_sbert.append({
                    'modismo': 'ERROR/OMITIDO',
                    'modelo': model,
                    'sbert_model': sbert_name,
                    'similarity': 0.0,
                    'definicion_real': '',
                    'definicion_generada': ''
                })

        # Calcular promedio sobre el total (GROUNTH_TRUTH)
        all_similarities = [r['similarity'] for r in resultados_sbert if r['modelo'] == model and r['sbert_model'] == sbert_name]
        sim_mean = np.mean(all_similarities)

        print(f"  • Similitud: {sim_mean:.4f}")
        print(f"    - Procesados: {registros_procesados}")
        print(f"    - Errores/omitidos: {registros_faltantes}")

    # Guardar resultados
    output_file = os.path.join(OUTPUT_DIR, f'prompt_2_{sbert_name.lower()}_sbert_similarity_resultados.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(resultados_sbert, f, ensure_ascii=False, indent=2)

    print()
    print(f"RESULTADOS con Sentence-BERT ({sbert_name}):")
    for model in MODEL_NAMES:
        model_results = [r for r in resultados_sbert if r['modelo'] == model]
        if model_results:
            similarities = [r['similarity'] for r in model_results]
            sim_mean = np.mean(similarities)
            sim_std = np.std(similarities)

            print(f"   {model}:")
            print(f"      • Similitud: {sim_mean:.4f} (±{sim_std:.4f}) [{len(model_results)}/{GROUNTH_TRUTH}]")

    print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 3. chrF Score
# ============================================================================
print(f"\n{'='*60}")
print(f"Calculando chrF Score")
print('='*60)

resultados_chrf = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")

    model_data = [d for d in data_p2_valid if d.get('modelo') == model]

    if not model_data:
        print(f"  ⚠ No hay datos para {model} - todos los registros con chrF 0")
        # Agregar todos los registros con chrF 0
        for _ in range(GROUNTH_TRUTH):
            resultados_chrf.append({
                'modismo': 'N/A',
                'modelo': model,
                'chrf_score': 0.0,
                'definicion_real': '',
                'definicion_generada': ''
            })
        print(f"  • chrF: 0.0000 (0/{GROUNTH_TRUTH})")
    else:
        referencias = [d['definicion_real'] for d in model_data]
        candidatos = [d['definicion_generada'] for d in model_data]

        # Calcular chrF
        chrf_scores = compute_chrf_batch(candidatos, referencias)

        for idx, score in enumerate(chrf_scores):
            resultados_chrf.append({
                'modismo': model_data[idx]['modismo'],
                'modelo': model,
                'chrf_score': float(score),
                'definicion_real': referencias[idx],
                'definicion_generada': candidatos[idx]
            })

    # Calcular registros faltantes (omitidos por errores)
    registros_procesados = len(model_data)
    registros_faltantes = GROUNTH_TRUTH - registros_procesados

    # Agregar registros faltantes con chrF 0
    if registros_faltantes > 0:
        print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan con chrF 0")
        for _ in range(registros_faltantes):
            resultados_chrf.append({
                'modismo': 'ERROR/OMITIDO',
                'modelo': model,
                'chrf_score': 0.0,
                'definicion_real': '',
                'definicion_generada': ''
            })

    # Calcular promedio sobre el total (GROUNTH_TRUTH)
    all_chrf_scores = [r['chrf_score'] for r in resultados_chrf if r['modelo'] == model]
    chrf_mean = np.mean(all_chrf_scores)

    print(f"  • chrF: {chrf_mean:.4f}")
    print(f"    - Procesados: {registros_procesados}")
    print(f"    - Errores/omitidos: {registros_faltantes}")

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_2_chrf_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_chrf, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS con chrF:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_chrf if r['modelo'] == model]
    if model_results:
        chrf_scores = [r['chrf_score'] for r in model_results]
        chrf_mean = np.mean(chrf_scores)
        chrf_std = np.std(chrf_scores)

        print(f"   {model}:")
        print(f"      • chrF: {chrf_mean:.4f} (±{chrf_std:.4f}) [{len(model_results)}/{GROUNTH_TRUTH}]")

print(f"\n✓ Guardado en: {output_file}")

print("\n" + "="*80)
print("PROMPT 2 COMPLETADO")
print("="*80 + "\n")

PROMPT 2: Modismo → Definición
Objetivo: Evaluar si el modelo puede generar una definición correcta del modismo
Métricas:
  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT
  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet
  - chrF: Character n-gram F-score
Nota: Los errores/omisiones se cuentan con score 0 en todas las métricas.
--------------------------------------------------------------------------------
Datos cargados: 136369 registros válidos de 136369 totales


Calculando BERTScore con BETO
Evaluando modelo: amazon/nova-micro-v1
  ⚠ 12 registros omitidos se cuentan con score 0
  • Precision: 0.4368
  • Recall:    0.4468
  • F1 Score:  0.4408
    - Procesados: 6521
    - Errores/omitidos: 12
Evaluando modelo: microsoft/phi-4
  ⚠ 381 registros omitidos se cuentan con score 0
  • Precision: 0.3993
  • Recall:    0.4168
  • F1 Score:  0.4069
    - Procesados: 6152
    - Errores/omitidos: 381
Evaluando modelo: amazon/nova-lite-v1
  ⚠ 75 reg

## 3. PROMPT 3: Modismo + Ejemplo → Literal + Definición

In [1]:
GROUNTH_TRUTH = 4_897

In [9]:
print("="*80)
print("PROMPT 3: Modismo + Ejemplo → Literal + Definición")
print("="*80)
print("Objetivo: Evaluar si el modelo genera interpretaciones literales correctas")
print("Métricas:")
print("  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT")
print("  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet")
print("  - chrF: Character n-gram F-score")
print("Nota: Los errores/omisiones se cuentan con score 0 en todas las métricas.")
print("-"*80)

# Cargar datos desde JSON
with open(os.path.join(DATA_DIR, 'prompt_3_metrics_data.json'), 'r', encoding='utf-8') as f:
    data_p3 = json.load(f)

# Filtrar datos con literal y definición generados
data_p3_valid = [d for d in data_p3 if d.get('significado_real') and d.get('definicion_generada')]

print(f"Datos cargados: {len(data_p3_valid)} registros válidos de {len(data_p3)} totales")
print()

# ============================================================================
# 1. BERTScore con BETO y SciBETO
# ============================================================================
bert_models = [
    ('BETO', compute_bertscore_beto),
    ('SciBETO', compute_bertscore_sci_beto)

]

for bert_name, bert_func in bert_models:
    print(f"\n{'='*60}")
    print(f"Calculando BERTScore con {bert_name}")
    print('='*60)

    resultados_p3 = []

    for model in MODEL_NAMES:
        print(f"Evaluando modelo: {model}")

        # Filtrar datos de este modelo
        model_data = [d for d in data_p3_valid if d.get('modelo') == model]

        if not model_data:
            print(f"  ⚠ No hay datos para {model} - todos los registros con score 0")
            # Agregar todos los registros con score 0
            for _ in range(GROUNTH_TRUTH):
                resultados_p3.append({
                    'modismo': 'N/A',
                    'ejemplo': '',
                    'modelo': model,
                    'bert_model': bert_name,
                    'precision': 0.0,
                    'recall': 0.0,
                    'f1_score': 0.0,
                    'significado_real': '',
                    'literal_generado': '',
                    'definicion_generada': ''
                })
            print(f"  • F1 Score: 0.0000 (0/{GROUNTH_TRUTH})")
            continue

        referencias = [d['significado_real'] for d in model_data]
        candidatos = [d['definicion_generada'] for d in model_data]

        # BERTScore: compara definición generada vs significado real
        P, R, F1 = bert_func(candidatos, referencias)

        for idx, (p, r, f1) in enumerate(zip(P.tolist(), R.tolist(), F1.tolist())):
            resultados_p3.append({
                'modismo': model_data[idx]['modismo'],
                'ejemplo': model_data[idx]['ejemplo'],
                'modelo': model,
                'bert_model': bert_name,
                'precision': p,
                'recall': r,
                'f1_score': f1,
                'significado_real': referencias[idx],
                'literal_generado': model_data[idx].get('literal_generado', ''),
                'definicion_generada': candidatos[idx]
            })

        # Calcular registros faltantes (omitidos por errores)
        registros_procesados = len(model_data)
        registros_faltantes = GROUNTH_TRUTH - registros_procesados

        # Agregar registros faltantes con score 0
        if registros_faltantes > 0:
            print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan con score 0")
            for _ in range(registros_faltantes):
                resultados_p3.append({
                    'modismo': 'ERROR/OMITIDO',
                    'ejemplo': '',
                    'modelo': model,
                    'bert_model': bert_name,
                    'precision': 0.0,
                    'recall': 0.0,
                    'f1_score': 0.0,
                    'significado_real': '',
                    'literal_generado': '',
                    'definicion_generada': ''
                })

        # Calcular promedios sobre el total (GROUNTH_TRUTH)
        all_f1_scores = [r['f1_score'] for r in resultados_p3 if r['modelo'] == model and r['bert_model'] == bert_name]
        all_precisions = [r['precision'] for r in resultados_p3 if r['modelo'] == model and r['bert_model'] == bert_name]
        all_recalls = [r['recall'] for r in resultados_p3 if r['modelo'] == model and r['bert_model'] == bert_name]

        f1_mean = np.mean(all_f1_scores)
        p_mean = np.mean(all_precisions)
        r_mean = np.mean(all_recalls)

        print(f"  • Precision: {p_mean:.4f}")
        print(f"  • Recall:    {r_mean:.4f}")
        print(f"  • F1 Score:  {f1_mean:.4f}")
        print(f"    - Procesados: {registros_procesados}")
        print(f"    - Errores/omitidos: {registros_faltantes}")

    # Guardar resultados
    output_file = os.path.join(OUTPUT_DIR, f'prompt_3_{bert_name.lower()}_bertscore_resultados.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(resultados_p3, f, ensure_ascii=False, indent=2)

    print()
    print(f"RESULTADOS con {bert_name}:")
    for model in MODEL_NAMES:
        model_results = [r for r in resultados_p3 if r['modelo'] == model]
        if model_results:
            f1_scores = [r['f1_score'] for r in model_results]
            precisions = [r['precision'] for r in model_results]
            recalls = [r['recall'] for r in model_results]

            f1_mean = np.mean(f1_scores)
            f1_std = np.std(f1_scores)
            p_mean = np.mean(precisions)
            r_mean = np.mean(recalls)

            print(f"   {model}:")
            print(f"      • Precision: {p_mean:.4f}")
            print(f"      • Recall:    {r_mean:.4f}")
            print(f"      • F1 Score:  {f1_mean:.4f} (±{f1_std:.4f}) [{len(model_results)}/{GROUNTH_TRUTH}]")

    print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 2. Sentence-BERT Similarity
# ============================================================================
sbert_models = [
    ('SciBETO-mean', compute_scibeto_similarity),
    ('paraphrase-mpnet', compute_sbert_similarity),
    ('XLM-RoBERTa', compute_xlm_similarity)

]

for sbert_name, sbert_func in sbert_models:
    print(f"\n{'='*60}")
    print(f"Calculando Sentence-BERT Similarity con {sbert_name}")
    print('='*60)

    resultados_sbert = []

    for model in MODEL_NAMES:
        print(f"Evaluando modelo: {model}")

        model_data = [d for d in data_p3_valid if d.get('modelo') == model]

        if not model_data:
            print(f"  ⚠ No hay datos para {model} - todos los registros con similarity 0")
            # Agregar todos los registros con similarity 0
            for _ in range(GROUNTH_TRUTH):
                resultados_sbert.append({
                    'modismo': 'N/A',
                    'ejemplo': '',
                    'modelo': model,
                    'sbert_model': sbert_name,
                    'similarity': 0.0,
                    'significado_real': '',
                    'literal_generado': '',
                    'definicion_generada': ''
                })
            print(f"  • Similitud: 0.0000 (0/{GROUNTH_TRUTH})")
            continue

        referencias = [d['significado_real'] for d in model_data]
        candidatos = [d['definicion_generada'] for d in model_data]

        # Calcular similitud con Sentence-BERT
        similarities = sbert_func(candidatos, referencias)

        for idx, sim in enumerate(similarities):
            resultados_sbert.append({
                'modismo': model_data[idx]['modismo'],
                'ejemplo': model_data[idx]['ejemplo'],
                'modelo': model,
                'sbert_model': sbert_name,
                'similarity': float(sim),
                'significado_real': referencias[idx],
                'literal_generado': model_data[idx].get('literal_generado', ''),
                'definicion_generada': candidatos[idx]
            })

        # Calcular registros faltantes (omitidos por errores)
        registros_procesados = len(model_data)
        registros_faltantes = GROUNTH_TRUTH - registros_procesados

        # Agregar registros faltantes con similarity 0
        if registros_faltantes > 0:
            print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan con similarity 0")
            for _ in range(registros_faltantes):
                resultados_sbert.append({
                    'modismo': 'ERROR/OMITIDO',
                    'ejemplo': '',
                    'modelo': model,
                    'sbert_model': sbert_name,
                    'similarity': 0.0,
                    'significado_real': '',
                    'literal_generado': '',
                    'definicion_generada': ''
                })

        # Calcular promedio sobre el total (GROUNTH_TRUTH)
        all_similarities = [r['similarity'] for r in resultados_sbert if r['modelo'] == model and r['sbert_model'] == sbert_name]
        sim_mean = np.mean(all_similarities)

        print(f"  • Similitud: {sim_mean:.4f}")
        print(f"    - Procesados: {registros_procesados}")
        print(f"    - Errores/omitidos: {registros_faltantes}")

    # Guardar resultados
    output_file = os.path.join(OUTPUT_DIR, f'prompt_3_{sbert_name.lower()}_sbert_similarity_resultados.json')
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(resultados_sbert, f, ensure_ascii=False, indent=2)

    print()
    print(f"RESULTADOS con Sentence-BERT ({sbert_name}):")
    for model in MODEL_NAMES:
        model_results = [r for r in resultados_sbert if r['modelo'] == model]
        if model_results:
            similarities = [r['similarity'] for r in model_results]
            sim_mean = np.mean(similarities)
            sim_std = np.std(similarities)

            print(f"   {model}:")
            print(f"      • Similitud: {sim_mean:.4f} (±{sim_std:.4f}) [{len(model_results)}/{GROUNTH_TRUTH}]")

    print(f"\n✓ Guardado en: {output_file}")

# ============================================================================
# 3. chrF Score
# ============================================================================
print(f"\n{'='*60}")
print(f"Calculando chrF Score")
print('='*60)

resultados_chrf = []

for model in MODEL_NAMES:
    print(f"Evaluando modelo: {model}")

    model_data = [d for d in data_p3_valid if d.get('modelo') == model]

    if not model_data:
        print(f"  ⚠ No hay datos para {model} - todos los registros con chrF 0")
        # Agregar todos los registros con chrF 0
        for _ in range(GROUNTH_TRUTH):
            resultados_chrf.append({
                'modismo': 'N/A',
                'ejemplo': '',
                'modelo': model,
                'chrf_score': 0.0,
                'significado_real': '',
                'literal_generado': '',
                'definicion_generada': ''
            })
        print(f"  • chrF: 0.0000 (0/{GROUNTH_TRUTH})")
        continue

    referencias = [d['significado_real'] for d in model_data]
    candidatos = [d['definicion_generada'] for d in model_data]

    # Calcular chrF
    chrf_scores = compute_chrf_batch(candidatos, referencias)

    for idx, score in enumerate(chrf_scores):
        resultados_chrf.append({
            'modismo': model_data[idx]['modismo'],
            'ejemplo': model_data[idx]['ejemplo'],
            'modelo': model,
            'chrf_score': float(score),
            'significado_real': referencias[idx],
            'literal_generado': model_data[idx].get('literal_generado', ''),
            'definicion_generada': candidatos[idx]
        })

    # Calcular registros faltantes (omitidos por errores)
    registros_procesados = len(model_data)
    registros_faltantes = GROUNTH_TRUTH - registros_procesados

    # Agregar registros faltantes con chrF 0
    if registros_faltantes > 0:
        print(f"  ⚠ {registros_faltantes} registros omitidos se cuentan con chrF 0")
        for _ in range(registros_faltantes):
            resultados_chrf.append({
                'modismo': 'ERROR/OMITIDO',
                'ejemplo': '',
                'modelo': model,
                'chrf_score': 0.0,
                'significado_real': '',
                'literal_generado': '',
                'definicion_generada': ''
            })

    # Calcular promedio sobre el total (GROUNTH_TRUTH)
    all_chrf_scores = [r['chrf_score'] for r in resultados_chrf if r['modelo'] == model]
    chrf_mean = np.mean(all_chrf_scores)

    print(f"  • chrF: {chrf_mean:.4f}")
    print(f"    - Procesados: {registros_procesados}")
    print(f"    - Errores/omitidos: {registros_faltantes}")

# Guardar resultados
output_file = os.path.join(OUTPUT_DIR, 'prompt_3_chrf_resultados.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(resultados_chrf, f, ensure_ascii=False, indent=2)

print()
print("RESULTADOS con chrF:")
for model in MODEL_NAMES:
    model_results = [r for r in resultados_chrf if r['modelo'] == model]
    if model_results:
        chrf_scores = [r['chrf_score'] for r in model_results]
        chrf_mean = np.mean(chrf_scores)
        chrf_std = np.std(chrf_scores)

        print(f"   {model}:")
        print(f"      • chrF: {chrf_mean:.4f} (±{chrf_std:.4f}) [{len(model_results)}/{GROUNTH_TRUTH}]")

print(f"\n✓ Guardado en: {output_file}")

print("\n" + "="*80)
print("PROMPT 3 COMPLETADO")
print("="*80 + "\n")


PROMPT 3: Modismo + Ejemplo → Literal + Definición
Objetivo: Evaluar si el modelo genera interpretaciones literales correctas
Métricas:
  - BERTScore (BETO y SciBETO): Similitud semántica con embeddings de BERT
  - Sentence-BERT: Similitud con modelo multilingüe paraphrase-mpnet
  - chrF: Character n-gram F-score
Nota: Los errores/omisiones se cuentan con score 0 en todas las métricas.
--------------------------------------------------------------------------------
Datos cargados: 98961 registros válidos de 99507 totales


Calculando BERTScore con BETO
Evaluando modelo: amazon/nova-micro-v1
  ⚠ 55 registros omitidos se cuentan con score 0
  • Precision: 0.4628
  • Recall:    0.4348
  • F1 Score:  0.4473
    - Procesados: 4842
    - Errores/omitidos: 55
Evaluando modelo: microsoft/phi-4
  ⚠ 51 registros omitidos se cuentan con score 0
  • Precision: 0.4718
  • Recall:    0.4792
  • F1 Score:  0.4743
    - Procesados: 4846
    - Errores/omitidos: 51
Evaluando modelo: amazon/nova-lite-v1
